In [64]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
url = "https://www.bfs.admin.ch/bfs/en/home/statistics/sustainable-development/monet-2030/all-indicators.html"

In [3]:
r = requests.get(url)

In [5]:
soup = BeautifulSoup(r.text)

In [54]:
monet_table_str = str(soup.find("wgl-indicator-set"))
pattern = r'"title":.*?}'
titles = re.findall(pattern, str(monet_table_str))
subtopics = [t[8:-1] for t in titles if "SDG" not in t]
topics = [t[8:-1] for t in titles if "SDG" in t]

In [82]:
sdgs = []

for t, st in zip(topics, subtopics):
    sdg_number = int(re.findall(r"\d+", t)[0])
    sdgs.append({"SDG": sdg_number,
                 "Topic": t.split(":")[-1].replace('"','').strip(),
                 "Subtopic": st.replace('"',''),
                 "Hyperlink Extension": "indicators/"+"-".join(st.replace('"','').lower().split())+".html"})

sdg_df = pd.DataFrame(sdgs)

In [93]:
sdg_df["SubtopicID"] = sdg_df.groupby("SDG").cumcount().add(1).astype(str)
sdg_df["ID"] = sdg_df.apply(lambda x: "MI" + str(x["SDG"]) + "." + x["SubtopicID"], axis=1)
sdg_df.drop("SubtopicID", axis=1, inplace=True)
sdg_df.set_index("ID")
sdg_df

,SDG,Topic,Subtopic,Hyperlink Extension,ID
0,1,No poverty,Poverty rate,indicators/poverty-rate.html,MI1.1
1,1,No poverty,Total social security expenditure,indicators/total-social-security-expenditure.html,MI1.2
2,1,No poverty,Fatalities caused by natural events,indicators/fatalities-caused-by-natural-events...,MI1.3
3,2,Zero hunger,Fruit and vegetables consumption,indicators/fruit-and-vegetables-consumption.html,MI2.1
4,2,Zero hunger,Nitrogen balance from agriculture,indicators/nitrogen-balance-from-agriculture.html,MI2.2
...,...,...,...,...,...
104,16,"Peace, justice and strong institutions",Language use: multilingualism,indicators/language-use:-multilingualism.html,MI16.7
105,16,"Peace, justice and strong institutions",Participation in cultural activities,indicators/participation-in-cultural-activitie...,MI16.8
106,16,"Peace, justice and strong institutions",Level of public debt,indicators/level-of-public-debt.html,MI16.9
107,17,Partnerships for the goals,Official Development Assistance,indicators/official-development-assistance.html,MI17.1
